In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [9]:
np.random.seed(42)
n_cases = 5000

data = {
    'Surgery_Type': np.random.choice(['عظام', 'جراحة عامة', 'رمد', 'قلب'], n_cases),
    'Total_Duration': np.random.randint(30, 300, n_cases), # وقت الغرفة الكلي
    'Gas_Usage_Min': np.random.randint(20, 280, n_cases),  # دقائق استخدام الغاز الفعلية
    'Actual_Surgery_Revenue': np.random.randint(10000, 80000, n_cases), # شامل الغازات
    'Supply_Cost': np.random.randint(1000, 5000, n_cases),
    'Medication_Cost': np.random.randint(500, 3000, n_cases),
    'Doctor_Fees': np.random.randint(3000, 15000, n_cases),
    'Anesthesia_Fee': 1500
}
df = pd.DataFrame(data)

In [10]:
df

,Surgery_Type,Total_Duration,Gas_Usage_Min,Actual_Surgery_Revenue,Supply_Cost,Medication_Cost,Doctor_Fees,Anesthesia_Fee
0,رمد,204,268,12076,1190,1743,12082,1500
1,قلب,268,276,74854,1139,1390,13683,1500
2,عظام,289,254,62342,1614,2681,5876,1500
3,رمد,192,93,47555,4302,2154,12823,1500
4,رمد,77,206,65208,4411,621,14895,1500
...,...,...,...,...,...,...,...,...
4995,عظام,196,233,26306,4565,2521,11482,1500
4996,جراحة عامة,236,134,17807,4107,1506,4909,1500
4997,رمد,94,68,61342,3871,714,10230,1500
4998,جراحة عامة,33,86,42143,2198,2665,4428,1500


In [11]:
# تأكيد أن دقائق الغاز لا تتعدى وقت العملية
df['Gas_Usage_Min'] = df[['Total_Duration', 'Gas_Usage_Min']].min(axis=1)

In [12]:
gas_supply_expenses = 150000    # فاتورة شراء الغازات
other_fixed_overhead = 500000   # إهلاك + تعقيم + رواتب

In [13]:
gas_rate = gas_supply_expenses / df['Gas_Usage_Min'].sum()

# معدل الإهلاك والتعقيم يعتمد على دقائق الغرفة الكلية
overhead_rate = other_fixed_overhead / df['Total_Duration'].sum()

# 4. تخصيص التكاليف (Allocation)
df['Allocated_Gas_Cost'] = df['Gas_Usage_Min'] * gas_rate
df['Allocated_Overhead_Cost'] = df['Total_Duration'] * overhead_rate

# 5. حساب صافي الربح (Net Profit)
df['Total_Costs'] = df['Supply_Cost'] + df['Medication_Cost'] + \
                     df['Doctor_Fees'] + df['Anesthesia_Fee'] + \
                     df['Allocated_Gas_Cost'] + df['Allocated_Overhead_Cost']

df['Net_Profit'] = df['Actual_Surgery_Revenue'] - df['Total_Costs']

In [15]:
X = df[['Total_Costs']] 

# 2. تحديد المتغير التابع (الإيراد الفعلي المدمج فيه الغازات)
y = df['Actual_Surgery_Revenue']

# 3. بناء وتدريب النموذج
model = LinearRegression()
model.fit(X, y)

# 4. استخراج النتائج
slope = model.coef_[0]      # معامل الارتباط (الميل)
intercept = model.intercept_ # الحد الثابت
r_sq = model.score(X, y)    # دقة العلاقة

print("--- 🔍 تحليل كفاءة التسعير (Cost vs Revenue) ---")
print(f"العائد المحقق مقابل كل 1 جنيه تكلفة: {slope:.2f} جنيهاً")
print(f"الإيراد الثابت الأساسي (Intercept): {intercept:.2f} جنيهاً")
print(f"دقة مطابقة الأسعار للتكاليف (R-squared): {r_sq*100:.2f}%")

--- 🔍 تحليل كفاءة التسعير (Cost vs Revenue) ---
العائد المحقق مقابل كل 1 جنيه تكلفة: -0.13 جنيهاً
الإيراد الثابت الأساسي (Intercept): 46662.07 جنيهاً
دقة مطابقة الأسعار للتكاليف (R-squared): 0.06%


In [17]:
final_output = profit_report.copy()

# تقريب الأرقام المالية (الإيرادات، التكاليف، الأرباح)
cols_to_format = ['Actual_Surgery_Revenue', 'Total_Costs', 'Net_Profit']
for col in cols_to_format:
    final_output[col] = final_output[col].map('{:,.2f}'.format)

# تنسيق النسب المئوية
final_output['Profit_Margin_%'] = final_output['Profit_Margin_%'].map('{:.2f}%'.format)

# تنسيق الدقائق (أرقام صحيحة مع فواصل)
final_output['Gas_Usage_Min'] = final_output['Gas_Usage_Min'].map('{:,}'.format)
final_output['Total_Duration'] = final_output['Total_Duration'].map('{:,}'.format)

# إعادة تسمية الأعمدة لتناسب تقرير مجلس الإدارة
final_output.columns = [
    'نوع الجراحة', 
    'إجمالي الإيرادات الفعلي', 
    'إجمالي التكاليف (مباشرة + ABC)', 
    'صافي الربح', 
    'دقائق استهلاك الغازات', 
    'إجمالي دقائق التشغيل', 
    'هامش الربح %'
]

# عرض الجدول النهائي المنظم
from IPython.display import display
print("--- 🏥 التقرير المالي الختامي لمراكز التكلفة - عام 2026 ---")
display(final_output)

--- 🏥 التقرير المالي الختامي لمراكز التكلفة - عام 2026 ---


,نوع الجراحة,إجمالي الإيرادات الفعلي,إجمالي التكاليف (مباشرة + ABC),صافي الربح,دقائق استهلاك الغازات,إجمالي دقائق التشغيل,هامش الربح %
2,عظام,"57,386,961.00","20,013,540.68","37,373,420.32","142,130","213,488",65.13%
3,قلب,"56,543,218.00","19,293,251.08","37,249,966.92","140,524","206,250",65.88%
0,جراحة عامة,"55,179,154.00","19,151,642.13","36,027,511.87","136,154","199,821",65.29%
1,رمد,"54,182,903.00","18,779,604.11","35,403,298.89","136,827","196,258",65.34%


In [20]:
import matplotlib.pyplot as plt

In [25]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# 1. تحديد المتغيرات المستقلة (X):
# نركز فقط على محركات التكلفة الزمنية (الموزعة بنظام ABC)
X = df[['Total_Duration', 'Gas_Usage_Min']]

# 2. تحديد المتغير التابع (Y): إجمالي التكاليف
# لا نزال نتنبأ بالتكلفة الكلية
y = df['Total_Costs']

# 3. بناء وتدريب النموذج
model_abc = LinearRegression()
model_abc.fit(X, y)

# 4. استخراج النتائج
intercept = model_abc.intercept_
coefficients = pd.DataFrame(model_abc.coef_, X.columns, columns=['Rate_Value'])

print("--- 🔍 نتائج تحليل الانحدار المعدل (تكاليف ABC فقط) ---")
print(f"التكلفة الثابتة لكل حالة (الحد الأدنى): {intercept:.2f} جنيهاً")
print("\nمعدل تغير التكلفة (المعاملات):")
print(coefficients)
print(f"\nدقة النموذج (R-squared): {model_abc.score(X, y)*100:.2f}%")


--- 🔍 نتائج تحليل الانحدار المعدل (تكاليف ABC فقط) ---
التكلفة الثابتة لكل حالة (الحد الأدنى): 15333.86 جنيهاً

معدل تغير التكلفة (المعاملات):
                Rate_Value
Total_Duration    0.391163
Gas_Usage_Min     0.449294

دقة النموذج (R-squared): 0.02%


In [26]:


# 1. تحديد المتغير المستقل (X): مدة العملية الكلية بالدقائق (Total_Duration)
X = df[['Total_Duration']]

# 2. تحديد المتغيرات التابعة (Y): الإيراد الإجمالي وصافي الربح
# سنقوم بعمل نموذجين للارتباط
y_revenue = df['Actual_Surgery_Revenue']
y_profit = df['Net_Profit']

# 3. بناء وتدريب النماذج
model_revenue = LinearRegression().fit(X, y_revenue)
model_profit = LinearRegression().fit(X, y_profit)

# 4. استخراج النتائج التحليلية
print("--- 🔍 تحليل الانحدار: المدة مقابل الإيراد والربح ---")
print(f"I. النموذج الأول (المدة vs الإيراد):")
print(f"  معامل الإيراد لكل دقيقة (Slope): {model_revenue.coef_[0]:.2f} جنيه")
print(f"  دقة النموذج (R-squared): {model_revenue.score(X, y_revenue)*100:.2f}%")

print(f"\nII. النموذج الثاني (المدة vs صافي الربح):")
print(f"  معامل الربح لكل دقيقة (Slope): {model_profit.coef_[0]:.2f} جنيه")
print(f"  دقة النموذج (R-squared): {model_profit.score(X, y_profit)*100:.2f}%")


--- 🔍 تحليل الانحدار: المدة مقابل الإيراد والربح ---
I. النموذج الأول (المدة vs الإيراد):
  معامل الإيراد لكل دقيقة (Slope): -5.38 جنيه
  دقة النموذج (R-squared): 0.04%

II. النموذج الثاني (المدة vs صافي الربح):
  معامل الربح لكل دقيقة (Slope): -5.97 جنيه
  دقة النموذج (R-squared): 0.05%


In [27]:
import pandas as pd
import numpy as np

# إعداد البذور العشوائية لضمان تكرار نفس النتائج في كل مرة
np.random.seed(100)
n_cases = 5000
data = []
surgery_types = ['عظام', 'جراحة عامة', 'رمد', 'قلب']

for _ in range(n_cases):
    s_type = np.random.choice(surgery_types)
    
    # تحديد مدة العملية وتكلفة المستلزمات والاطباء بناءً على نوع الجراحة (منطقي)
    if s_type == 'عظام':
        dur = np.random.randint(60, 180)
        supp_cost = np.random.randint(2000, 4000)
        doc_fees = np.random.randint(6000, 10000)
    elif s_type == 'جراحة عامة':
        dur = np.random.randint(45, 120)
        supp_cost = np.random.randint(1500, 3000)
        doc_fees = np.random.randint(4000, 7000)
    elif s_type == 'رمد':
        dur = np.random.randint(20, 60)
        supp_cost = np.random.randint(1000, 2000)
        doc_fees = np.random.randint(3000, 5000)
    else: # قلب
        dur = np.random.randint(120, 300)
        supp_cost = np.random.randint(4000, 6000)
        doc_fees = np.random.randint(8000, 15000)
        
    # تحديد الإيراد الفعلي بناءً على التكلفة + هامش ربح معقول
    actual_rev = supp_cost * np.random.uniform(3.0, 5.0) # يضمن علاقة منطقية بين التكلفة والإيراد
    
    data.append({
        'Surgery_Type': s_type,
        'Total_Duration': dur, # وقت الغرفة الكلي (دقائق)
        'Gas_Usage_Min': dur * np.random.uniform(0.8, 0.95), # دقائق الغاز (منطقياً أقل من وقت الغرفة)
        'Actual_Surgery_Revenue': actual_rev, # الإيراد الفعلي (شامل الغازات)
        'Supply_Cost': supp_cost, # تكلفة المستلزمات المباشرة
        'Medication_Cost': np.random.randint(500, 3000),
        'Doctor_Fees': doc_fees,
        'Anesthesia_Fee': 1500 # أتعاب تخدير ثابتة
    })

df = pd.DataFrame(data)

# طباعة أول 5 صفوف للتحقق
print("تم إنشاء جدول البيانات df بنجاح (5000 سجل منطقي).")
print(df.head())


تم إنشاء جدول البيانات df بنجاح (5000 سجل منطقي).
  Surgery_Type  Total_Duration  Gas_Usage_Min  Actual_Surgery_Revenue  \
0         عظام              84      77.844179            14853.426765   
1         عظام             158     137.129936             7021.702390   
2          قلب             227     181.798075            22025.324107   
3          قلب             228     210.314886            19809.157119   
4         عظام             151     141.638539            17045.757059   

   Supply_Cost  Medication_Cost  Doctor_Fees  Anesthesia_Fee  
0         3859              850         9943            1500  
1         2053              780         7602            1500  
2         5340             2934        10960            1500  
3         5924             1112        10463            1500  
4         3979             2865         8883            1500  
